In [286]:
import shutup; shutup.please()

In [287]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
from datetime import datetime
import numpy as np
from prefixspan import PrefixSpan
from mlxtend.frequent_patterns import fpgrowth, apriori
from mlxtend.preprocessing import TransactionEncoder


In [288]:
path = "/Users/ehambardzumy/Desktop/mimic_raw_data2.csv"

In [289]:
df = pd.read_csv(path) 

In [9]:
df = df.dropna()
df['charttime'] = pd.to_datetime(df['charttime'])
df['hadm_id'] = df['hadm_id'].astype(int)
# combine admission with subject_ids
df['subject_id'] = df['subject_id'].astype("str")+"_"+df['hadm_id'].astype("str")
df = df.drop_duplicates()

# Generate unique 5-length digit IDs for each string
unique_strings = df.query("tbl_name=='pharmacy'")['col_id'].drop_duplicates().tolist()
id_mapping = {string: f"{index:05}" for index, string in enumerate(unique_strings)}
reverse_id_mapping = {v: k for k, v in id_mapping.items()}


# apply the mapping replacing pharmacy string with the ids
df.loc[df['tbl_name'] == 'pharmacy', 'col_id'] = df['col_id'].map(id_mapping)
df['col_id'] = df['col_id'].astype("str")


# create a prefix based on 3 characters of each tbl_name
df['prefix'] = df['tbl_name'].apply(lambda x: x[0:3]+'_')
# add the prefix to the col_id
df['col_id'] = df['prefix']+df['col_id']

In [10]:
df = df.drop(columns=["hadm_id", "prefix"])

In [11]:
# add all items to a list 
df = (df.groupby(['subject_id', 'charttime'])['col_id'].apply(list)
        .reset_index(name='col_id_list'))

In [12]:
df = df.sort_values(['subject_id','charttime'], ascending=[True, True])

###

In [13]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(df[ "col_id_list"].to_list()).transform(df[ "col_id_list"].to_list())


In [14]:
onehot = pd.DataFrame(te_ary, columns=te.columns_)


In [15]:
onehot

,lab_50801,lab_50802,lab_50803,lab_50804,lab_50805,lab_50806,lab_50808,lab_50809,lab_50810,lab_50811,...,pro_B410YZZ,pro_B4141ZZ,pro_B4151ZZ,pro_B41DYZZ,pro_B41FYZZ,pro_B51MYZA,pro_B543ZZA,pro_B547ZZA,pro_BF10YZZ,pro_BF13YZZ
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35662,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35663,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35664,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35665,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


####--

In [22]:
patterns = fpgrowth(onehot, min_support=0.15, use_colnames=True)
patterns['itemsets'] = patterns['itemsets'].apply(lambda x: frozenset(x))

In [132]:
dict_pattern_freq = patterns.set_index('itemsets')['support'].to_dict()
dict_keys = set(dict_pattern_freq.keys())


In [27]:
from itertools import chain, combinations

def powerset(iterable, size):
    iterable_list = list(iterable)
    return combinations(iterable_list, size)

def get_pattern_support(pattern, dict_pattern_freq=dict_pattern_freq, dict_keys=dict_keys):
    if pattern in dict_keys:
        support = dict_pattern_freq[pattern]
    else:
        support = 0
    return support



In [77]:
def break_down(pattern):
    best_subsets = []
    not_converged = True
    size_iteration = len(pattern)-1

    while size_iteration>0:
        print(size_iteration)
        subset_fixed_len = powerset(pattern, size=size_iteration)
        arg_max = None
        max_val = 0
        for subset in subset_fixed_len:
            fr_subset = frozenset(subset)
            support = get_pattern_support(frozenset(subset))
            if support>max_val:
                max_val=support
                arg_max=fr_subset
        if max_val!=0:
            print(arg_max)
            best_subsets.append(arg_max)
            pattern = pattern.difference(arg_max)
            size_iteration -= 1
        else:
            size_iteration -= 1
    if not best_subsets:
        best_subsets.append(pattern)
        
    return best_subsets

In [79]:
pattern = frozenset({'lab_50931', 'lab_51275', 'lab_50912', 'lab_51248', 'lab_50878', 'lab_50971', 'lab_51222', 'lab_50863', 'lab_50885', 'lab_51249', 'lab_50983', 'lab_51274', 'lab_50960', 'lab_50893', 'lab_51250', 'lab_50902', 'lab_50970', 'lab_50861', 'lab_51277', 'lab_50862', 'lab_51006', 'lab_51301', 'lab_51237', 'lab_51221', 'lab_51279', 'lab_50882', 'lab_50868', 'lab_51265'})


In [80]:
break_down(pattern)

27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
frozenset({'lab_51265', 'lab_51222', 'lab_51221', 'lab_51250', 'lab_51279', 'lab_51277', 'lab_51249', 'lab_51248', 'lab_51301'})
8
frozenset({'lab_50983', 'lab_50882', 'lab_50902', 'lab_50931', 'lab_51006', 'lab_50971', 'lab_50868', 'lab_50912'})
7
6
5
4
3
2
1
frozenset({'lab_50960'})


[frozenset({'lab_51221',
            'lab_51222',
            'lab_51248',
            'lab_51249',
            'lab_51250',
            'lab_51265',
            'lab_51277',
            'lab_51279',
            'lab_51301'}),
 frozenset({'lab_50868',
            'lab_50882',
            'lab_50902',
            'lab_50912',
            'lab_50931',
            'lab_50971',
            'lab_50983',
            'lab_51006'}),
 frozenset({'lab_50960'})]

In [290]:
buckets

[[frozenset({'lab_51221',
             'lab_51222',
             'lab_51248',
             'lab_51249',
             'lab_51250',
             'lab_51265',
             'lab_51277',
             'lab_51279',
             'lab_51301'}),
  frozenset({'lab_50868',
             'lab_50882',
             'lab_50902',
             'lab_50912',
             'lab_50931',
             'lab_50960',
             'lab_50971',
             'lab_50983',
             'lab_51006'})],
 [frozenset({'lab_50868',
             'lab_50882',
             'lab_50902',
             'lab_50912',
             'lab_50931',
             'lab_50971',
             'lab_50983',
             'lab_51006'}),
  frozenset({'lab_51221',
             'lab_51222',
             'lab_51248',
             'lab_51249',
             'lab_51250',
             'lab_51277',
             'lab_51279',
             'lab_51301'}),
  frozenset({'lab_51221',
             'lab_51222',
             'lab_51248',
             'lab_51249',
   

In [ ]:

def break_down(pattern, buckets):
    best_subsets = []
    for B in buckets:
        if len(B[0])<len(pattern):
            for SE in B:
                if SE.issubset(pattern):
                    best_subsets.append(SE)
                    pattern = pattern.difference(SE)
                    if not pattern:
                         return best_subsets
    if pattern:
        best_subsets.append(pattern)
        
    return best_subsets 

In [133]:
patterns = fpgrowth(onehot, min_support=0.15, use_colnames=True)
patterns['itemsets'] = patterns['itemsets'].apply(lambda x: frozenset(x))
patterns['length'] = patterns['itemsets'].apply(len)
patterns = patterns.sort_values(["length", "support"],ascending=[False, False])

In [221]:
patterns

,support,itemsets,length
766,0.165083,"(lab_51265, lab_51222, lab_51221, lab_51250, l...",9
1021,0.152746,"(lab_50983, lab_50882, lab_50902, lab_50960, l...",9
264,0.169092,"(lab_50983, lab_50882, lab_50902, lab_50931, l...",8
639,0.165279,"(lab_51222, lab_51221, lab_51250, lab_51279, l...",8
511,0.165111,"(lab_51265, lab_51222, lab_51221, lab_51250, l...",8
759,0.165111,"(lab_51265, lab_51222, lab_51221, lab_51250, l...",8
760,0.165083,"(lab_51265, lab_51222, lab_51221, lab_51279, l...",8
761,0.165083,"(lab_51265, lab_51222, lab_51221, lab_51250, l...",8
762,0.165083,"(lab_51265, lab_51222, lab_51221, lab_51250, l...",8
763,0.165083,"(lab_51265, lab_51222, lab_51221, lab_51250, l...",8


In [135]:
buckets = (patterns
           .groupby(['length'])['itemsets'].apply(list).reset_index(name='buckets')
          ).sort_values("length",ascending=False)['buckets'].to_list()       

In [292]:
buckets[0]

[frozenset({'lab_51221',
            'lab_51222',
            'lab_51248',
            'lab_51249',
            'lab_51250',
            'lab_51265',
            'lab_51277',
            'lab_51279',
            'lab_51301'}),
 frozenset({'lab_50868',
            'lab_50882',
            'lab_50902',
            'lab_50912',
            'lab_50931',
            'lab_50960',
            'lab_50971',
            'lab_50983',
            'lab_51006'})]

In [218]:
pattern = frozenset({'lab_50931', 'lab_51275', 'lab_50912', 'lab_51248', 'lab_50878', 'lab_50971', 'lab_51222', 'lab_50863', 'lab_50885', 'lab_51249', 'lab_50983', 'lab_51274', 'lab_50960', 'lab_50893', 'lab_51250', 'lab_50902', 'lab_50970', 'lab_50861', 'lab_51277', 'lab_50862', 'lab_51006', 'lab_51301', 'lab_51237', 'lab_51221', 'lab_51279', 'lab_50882', 'lab_50868', 'lab_51265'})

In [ ]:
[frozenset({'lab_50868',
            'lab_50882',
            'lab_50902',
            'lab_50912',
            'lab_50931',
            'lab_50971',
            'lab_50983',
            'lab_51006'}),
 frozenset({'lab_50960'})]

In [219]:
break_down(pattern, buckets)

[frozenset({'lab_51221',
            'lab_51222',
            'lab_51248',
            'lab_51249',
            'lab_51250',
            'lab_51265',
            'lab_51277',
            'lab_51279',
            'lab_51301'}),
 frozenset({'lab_50868',
            'lab_50882',
            'lab_50902',
            'lab_50912',
            'lab_50931',
            'lab_50960',
            'lab_50971',
            'lab_50983',
            'lab_51006'}),
 frozenset({'lab_50861',
            'lab_50862',
            'lab_50863',
            'lab_50878',
            'lab_50885',
            'lab_50893',
            'lab_50970',
            'lab_51237',
            'lab_51274',
            'lab_51275'})]

In [ ]:
df['length_items'] = df['col_id_list'].apply(len)
unique_sequences = df.query("length_items>1").col_id_list.drop_duplicates().apply(lambda x: frozenset(x)).to_list()



In [301]:
## After obtaining patterns we can evaluate if they are very similar or not and post process them??

In [ ]:
## Chalanges
# - groupping of items and large basket breakdown
# - closed pattern mining using prefix span
# - closed pattern mining using timestamp info
# - filterning very similar patterns
# - spv generation

---

In [ ]:
# from itertools import chain, combinations

# def powerset(iterable):
#     s = list(iterable)
#     return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


# from itertools import chain, combinations

# def powerset(iterable):
#     s = list(iterable)
#     return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# # I could sort the dict of supports by support

# # btw does not work for large sets
# def break_down(pattern):
#     best_subsets = []
#     #return all subsets of the pattern, largest coming first
#     all_subsets = sorted(powerset(pattern), key=len, reverse=True)
#     for size in reversed(range(1,len(pattern))):
#         print(size)
#         #subset of that specific size
#         subset_fixed_len = [frozenset(subset) for subset in all_subsets if len(subset)==size]
#         print(subset_fixed_len)
#         # supports for that specific size 
#         #supports = [dict_pattern_freq[subset] for subset in subset_fixed_len]
#         supports = [dict_pattern_freq[subset] for subset in subset_fixed_len]
#         # if not empty
#         if supports:
#             arg_max = np.argmax(np.array(supports))
#             # obtain most frequest one
#             best_longest_subset = subset_fixed_len[arg_max]
#             print(best_longest_subset)
#             best_subsets.append(best_longest_subset)
#             pattern = pattern.difference(best_longest_subset)
#             if len(pattern)==1: 
#                 best_subsets.append(pattern)
#                 break
#             else:
#                 all_subsets = sorted(powerset(pattern), key=len, reverse=True)
#                 print(all_subsets)
#     return best_subsets


# I don't need prefixspan support I can use apriori
# def break_down(pattern):
#     best_subsets = []
#     not_converged = True
#     size_iteration = len(pattern)-1
#     while not_converged:
#         print(size_iteration)

#         try:
#             print("yes")
#             subset_fixed_len = list(powerset(pattern, size=size_iteration))
#             subset_fixed_len = [frozenset(subset) for subset in subset_fixed_len]
#             print(subset_fixed_len)
#             supports = [dict_pattern_freq[subset] for subset in subset_fixed_len]
#             arg_max = np.argmax(np.array(supports))
#             # obtain most frequest one
#             best_longest_subset = subset_fixed_len[arg_max]
#             best_subsets.append(best_longest_subset)
#             pattern = pattern.difference(best_longest_subset)
#             print(size_iteration, size_iteration, size_iteration)
#             if size_iteration==1: 
#                 best_subsets.append(pattern)
#                 not_converged = True
#             size_iteration = size_iteration-1
#         except KeyError:
#                 print("no")
#                 if size_iteration==1: 
#                     best_subsets.append(pattern)
#                     not_converged = True
#                 size_iteration = size_iteration-1

            
#     return best_subsets


# def break_down(pattern):
#     best_subsets = []
#     not_converged = True
#     size_iteration = len(pattern)-1

#     while size_iteration>0:
#         print(size_iteration)
#         subset_fixed_len = list(powerset(pattern, size=size_iteration))
#         supports = []
#         supports = [get_pattern_support(frozenset(subset)) for subset in subset_fixed_len]
#         print("three")
#         supports = np.array(supports)
#         if np.sum(supports)!=0:
#             arg_max = np.argmax(np.array(supports))
#             best_longest_subset = subset_fixed_len[arg_max]
#             best_subsets.append(best_longest_subset)
#             pattern = pattern.difference(best_longest_subset)
#             size_iteration -= 1
#         else:
#             size_iteration -= 1
#     if not best_subsets:
#         best_subsets.append(pattern)
        
#     return best_subsets

In [251]:
from spmf import Spmf

In [249]:
input_example_list = [
    [[1], [1, 2, 3], [1, 3], [4], [3, 6]],
    [[1, 4], [3], [2, 3], [1, 5]],
    [[5, 6], [1, 2], [4, 6], [3], [2]],
    [[5], [7], [1, 6], [3], [2], [3]]
]

In [282]:
spmf = Spmf("PrefixSpan", input_direct=input_example_list,
             arguments=[0.2,3, True])

In [283]:
spmf.run()
# print(spmf.parse_output())
print(spmf.to_pandas_dataframe(pickle=True))


>/Users/ehambardzumy/Documents/app-core-analytics/spmf.jar
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 12 ms
 Frequent sequences count : 195
 Max memory (mb) : 10.240081787109375
 minsup = 1 sequences.
 Pattern count : 195


       pattern  sup
0          [1]    4
1        [1 2]    2
2      [1 2 3]    1
3     [1 2, 1]    1
4     [1 2, 2]    1
5     [1 2, 3]    2
6     [1 2, 4]    2
7     [1 2, 6]    2
8        [1 3]    1
9     [1 3, 1]    1
10    [1 3, 3]    1
11    [1 3, 4]    1
12    [1 3, 6]    1
13       [1 4]    1
14    [1 4, 1]    1
15    [1 4, 2]    1
16    [1 4, 3]    1
17    [1 4, 5]    1
18       [1 5]    1
19       [1 6]    1
20    [1 6, 2]    1
21    [1 6, 3]    1
22      [1, 1]    2
23    [1, 1 2]    1
24    [1, 1 3]    1
25    [1, 1 5]    1
26   [1, 1, 1]    1
27   [1, 1, 3]    1
28   [1, 1, 4]    1
29   [1, 1, 6]    1
30      [1, 2]    4
31    [1, 2 3]    2
32   [1, 2, 1]    2
33   [1, 2, 3]    2
34   [1, 2, 4]    1
35   [1, 2, 5]    1
36

In [285]:
[frozenset({'entered_hospoital'}), frozenset({'lab_test1'}), frozenset({'exited_hopital'})]

[frozenset({'entered_hospoital'}),
 frozenset({'lab_test1'}),
 frozenset({'exited_hopital'})]

In [ ]:
# Sample sequential data
sequences = [
    [1, 2, 3],
    [1, 2, 4],
    [1, 3],
    [2, 4],
    [2, 3],
    [1, 2, 3, 4]
]



In [244]:
>>> from pymining import seqmining
>>> seqs = ( 'caabc', 'abcb', 'cabc', 'abbca')
>>> freq_seqs = seqmining.freq_seq_enum(seqs, 2)
>>> sorted(freq_seqs)


[(('a',), 4),
 (('a', 'a'), 2),
 (('a', 'b'), 4),
 (('a', 'b', 'b'), 2),
 (('a', 'b', 'c'), 4),
 (('a', 'c'), 4),
 (('b',), 4),
 (('b', 'b'), 2),
 (('b', 'c'), 4),
 (('c',), 4),
 (('c', 'a'), 3),
 (('c', 'a', 'b'), 2),
 (('c', 'a', 'b', 'c'), 2),
 (('c', 'a', 'c'), 2),
 (('c', 'b'), 3),
 (('c', 'b', 'c'), 2),
 (('c', 'c'), 2)]

In [242]:
report

{frozenset({'a'}): 3,
 frozenset({'a', 'c'}): 2,
 frozenset({'b'}): 4,
 frozenset({'b', 'c'}): 3,
 frozenset({'c'}): 4}

In [245]:
[frozenset({'a'}), frozenset({'a', "b"})]

[frozenset({'a'}), frozenset({'a', 'b'})]